In [211]:
library(hash)
library(dplyr)

In [213]:
my_db <- src_sqlite('database.sqlite', create = FALSE)

Get all the players. Btw you can get Kaggle Soceer DB from https://www.kaggle.com/hugomathien/soccer 

Credit for ordering a dataframe based on a column : http://www.statmethods.net/management/sorting.html

Credit for reading data from a database : https://cran.rstudio.com/web/packages/dplyr/vignettes/introduction.html

Credit for merging 2 dataframes : http://www.statmethods.net/management/merging.html

In [238]:
player_data_frame <- as.data.frame(player <- tbl(my_db, sql("SELECT player_api_id,player_name FROM Player")))

Get all the rankings from player_attribute table and sort them 


In [239]:
sql_stat <- "Select l.overall_rating as overall_rating, p.player_api_id from Player_Attributes l inner join Player p where l.player_api_id=p.player_api_id"
league_id_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))
newdata <- league_id_data_frame[order(-league_id_data_frame$overall_rating),] 

Since there are multiple rankings for many players, we'll take the highest rankings of each player. This step might take some time to finish

In [240]:
player_rank_map <- hash()
rows_to_be_deleted <- hash()
 i=1
 length <- nrow(newdata)
 
 while(i<= length){
 player_api_id <- as.character(newdata[i,]$player_api_id)
 
 if(has.key(player_api_id,player_rank_map)){
    rows_to_be_deleted[[as.character(i)]] <- 1
     
} else {
    player_rank_map[[player_api_id]] <- 1 
 }
i = i+1
}

newdata <- newdata[ -c(as.numeric(keys(rows_to_be_deleted))),]

Append rankings to player data frame

In [241]:
player_data_frame <- merge(player_data_frame,newdata,by="player_api_id",all.x=TRUE)

Get the GoalKeepers 

In [242]:
sql_stat <- "select p.player_api_id, l.gk_diving+l.gk_handling+l.gk_kicking+l.gk_positioning+l.gk_reflexes as gk_attributes_sum from Player_Attributes l inner join Player p where p.player_api_id=l.player_api_id and gk_diving is not null and gk_handling is not null and  gk_kicking is not null and  gk_positioning is not null and  gk_reflexes is not null"
goalie_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))
goalie_data_frame <- goalie_data_frame[order(-goalie_data_frame$gk_attributes_sum),]

Since there are multiple gk_attributes_sum for many players, we'll take the highest rankings of each player. This step might take some time to finish

In [243]:
player_rank_map <- hash()
rows_to_be_deleted <- hash()
 i=1
 length <- nrow(goalie_data_frame)
 
 while(i<= length){
 player_api_id <- as.character(goalie_data_frame[i,]$player_api_id)
 
 if(has.key(player_api_id,player_rank_map)){
    rows_to_be_deleted[[as.character(i)]] <- 1
     
} else {
    player_rank_map[[player_api_id]] <- 1 
 }
i = i+1
}

goalie_data_frame <- goalie_data_frame[ -c(as.numeric(keys(rows_to_be_deleted))),]

Append gk_attributes to player data frame

In [244]:
player_data_frame <- merge(player_data_frame,goalie_data_frame,by="player_api_id",all.x=TRUE)

Get the matches

In [245]:
 sql_stat <- "Select count(l.id) as match_count, p.player_api_id from Match l inner join Player p where l.home_player_1=p.player_api_id or  l.home_player_2=p.player_api_id or  l.home_player_3=p.player_api_id or  l.home_player_4=p.player_api_id or  l.home_player_5=p.player_api_id or  l.home_player_6=p.player_api_id or  l.home_player_7=p.player_api_id or  l.home_player_8=p.player_api_id or  l.home_player_9=p.player_api_id or  l.home_player_10=p.player_api_id or l.home_player_11=p.player_api_id or l.away_player_1=p.player_api_id or  l.away_player_2=p.player_api_id or  l.away_player_3=p.player_api_id or  l.away_player_4=p.player_api_id or  l.away_player_5=p.player_api_id or  l.away_player_6=p.player_api_id or  l.away_player_7=p.player_api_id or  l.away_player_8=p.player_api_id or  l.away_player_9=p.player_api_id or  l.away_player_10=p.player_api_id or l.away_player_11=p.player_api_id group by p.player_api_id"
 match_id_data_frame <- as.data.frame(player <- tbl(my_db, sql(sql_stat)))


Merge Data

In [246]:
player_data_frame <- merge(player_data_frame,match_id_data_frame,by="player_api_id",all.x=TRUE)

Get the leagues

In [247]:
 sql_stat <- "Select count(distinct(l.league_id)) as league_count, p.player_api_id from Match l inner join Player p where l.home_player_1=p.player_api_id or  l.home_player_2=p.player_api_id or  l.home_player_3=p.player_api_id or  l.home_player_4=p.player_api_id or  l.home_player_5=p.player_api_id or  l.home_player_6=p.player_api_id or  l.home_player_7=p.player_api_id or  l.home_player_8=p.player_api_id or  l.home_player_9=p.player_api_id or  l.home_player_10=p.player_api_id or l.home_player_11=p.player_api_id or l.away_player_1=p.player_api_id or  l.away_player_2=p.player_api_id or  l.away_player_3=p.player_api_id or  l.away_player_4=p.player_api_id or  l.away_player_5=p.player_api_id or  l.away_player_6=p.player_api_id or  l.away_player_7=p.player_api_id or  l.away_player_8=p.player_api_id or  l.away_player_9=p.player_api_id or  l.away_player_10=p.player_api_id or l.away_player_11=p.player_api_id group by p.player_api_id"
league_id_data_frame <- as.data.frame(player <- tbl(my_db, sql(sql_stat)))

Merge Data

In [248]:
player_data_frame <- merge(player_data_frame,league_id_data_frame,by="player_api_id",all.x=TRUE)

******** Player Finished ******* **** Team Begins ******

Get Team build up attributes and Merge them

In [249]:
team_data_frame <- as.data.frame(player <- tbl(my_db, sql("SELECT team_api_id,team_long_name FROM Team")))

In [250]:
sql_stat <- "select team_api_id, buildUpPlaySpeed+buildUpPlaySpeedClass+buildUpPlayDribbling+buildUpPlayDribblingClass+buildUpPlayPassing+buildUpPlayPassingClass+buildUpPlayPositioningClass as buildup_attributes_sum from Team_Attributes where  buildUpPlaySpeed is not null and buildUpPlaySpeedClass is not null and  buildUpPlayDribbling is not null and  buildUpPlayDribblingClass is not null and  buildUpPlayPassing  is not null  and buildUpPlayPassingClass is not null and buildUpPlayPositioningClass is not null "
team_build_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))
team_build_data_frame <- team_build_data_frame[order(-team_build_data_frame$buildup_attributes_sum),]

In [251]:
rank_map <- hash()
rows_to_be_deleted <- hash()
 i=1
 length <- nrow(team_build_data_frame)
 
 while(i<= length){
 team_api_id <- as.character(team_build_data_frame[i,]$team_api_id)
 
 if(has.key(team_api_id,rank_map)){
    rows_to_be_deleted[[as.character(i)]] <- 1
     
} else {
    rank_map[[team_api_id]] <- 1 
 }
i = i+1
}

team_build_data_frame <- team_build_data_frame[ -c(as.numeric(keys(rows_to_be_deleted))),]
team_data_frame <- merge(team_data_frame,team_build_data_frame,by="team_api_id",all.x=TRUE)


Get Team chance creation and merge them. I hate copy pasting :)

In [252]:
sql_stat <- "select team_api_id, chanceCreationPassing + chanceCreationPassingClass + chanceCreationCrossing + chanceCreationCrossingClass + chanceCreationShooting + chanceCreationShootingClass + chanceCreationPositioningClass as chance_attributes_sum from Team_Attributes where  chanceCreationPassing is not null and chanceCreationPassingClass is not null and  chanceCreationCrossing is not null and  chanceCreationCrossingClass is not null and  chanceCreationShooting  is not null  and chanceCreationShootingClass is not null and chanceCreationPositioningClass is not null "
team_chance_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))

In [253]:
rank_map <- hash()
rows_to_be_deleted <- hash()
 i=1
 length <- nrow(team_chance_data_frame)
 
 while(i<= length){
 team_api_id <- as.character(team_chance_data_frame[i,]$team_api_id)
 
 if(has.key(team_api_id,rank_map)){
    rows_to_be_deleted[[as.character(i)]] <- 1
     
} else {
    rank_map[[team_api_id]] <- 1 
 }
i = i+1
}

team_chance_data_frame <- team_chance_data_frame[ -c(as.numeric(keys(rows_to_be_deleted))),]
team_data_frame <- merge(team_data_frame,team_chance_data_frame,by="team_api_id",all.x=TRUE)


Get Defence attributes

In [254]:
sql_stat <- "select team_api_id, defencePressure + defencePressureClass + defenceAggression + defenceAggressionClass + defenceTeamWidth + defenceTeamWidthClass + defenceDefenderLineClass as defence_attributes_sum from Team_Attributes where  defencePressure is not null and defencePressureClass is not null and  defenceAggression is not null and  defenceAggressionClass is not null and  defenceTeamWidth  is not null  and defenceTeamWidthClass is not null and defenceDefenderLineClass is not null "
team_defence_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))

In [255]:
rank_map <- hash()
rows_to_be_deleted <- hash()
 i=1
 length <- nrow(team_defence_data_frame)
 
 while(i<= length){
 team_api_id <- as.character(team_defence_data_frame[i,]$team_api_id)
 
 if(has.key(team_api_id,rank_map)){
    rows_to_be_deleted[[as.character(i)]] <- 1
     
} else {
    rank_map[[team_api_id]] <- 1 
 }
i = i+1
}

team_defence_data_frame <- team_defence_data_frame[ -c(as.numeric(keys(rows_to_be_deleted))),]
team_data_frame <- merge(team_data_frame,team_defence_data_frame,by="team_api_id",all.x=TRUE)

Get Players Fielded. Get players fielded by team as home team and as away team and join them

In [256]:
sql_stat <- "select ifnull(home_player_1,-1) as player_1,ifnull(home_player_2,-1) as player_2,ifnull(home_player_3,-1) as player_3,ifnull(home_player_4,-1) as player_4,ifnull(home_player_5,-1) player_5,ifnull(home_player_6,-1) player_6,ifnull(home_player_7,-1) player_7,ifnull(home_player_8,-1) player_8,ifnull(home_player_9,-1) player_9,ifnull(home_player_10,-1) player_10,ifnull(home_player_11,-1) player_11, home_team_api_id as team_api_id from Match"
matches_home_player_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))

sql_stat <- "select ifnull(away_player_1,-1) as player_1,ifnull(away_player_2,-1) as player_2,ifnull(away_player_3,-1) as player_3,ifnull(away_player_4,-1) as player_4,ifnull(away_player_5,-1) player_5,ifnull(away_player_6,-1) player_6,ifnull(away_player_7,-1) player_7,ifnull(away_player_8,-1) player_8,ifnull(away_player_9,-1) player_9,ifnull(away_player_10,-1) player_10,ifnull(away_player_11,-1) player_11, away_team_api_id as team_api_id from Match"
matches_away_player_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))
matches_player_data_frame <- bind_rows(matches_home_player_data_frame,matches_away_player_data_frame)

In [257]:
sql_stat <- "Select team_api_id from Team"
team_id_data_frame <- as.data.frame(tbl(my_db, sql(sql_stat)))

In [258]:
i <-1

team_id_data_frame["players_fielded"] <- 0
length_teams <- nrow(team_id_data_frame)

while(i < length_teams){
    count<-0
    
    df_this_team <- matches_player_data_frame[matches_player_data_frame$team_api_id %in% team_id_data_frame[i,1],]
    
    count<- n_distinct(c(df_this_team$player_1,df_this_team$player_2,df_this_team$player_3,df_this_team$player_4,df_this_team$player_5,df_this_team$player_6,df_this_team$player_7,df_this_team$player_8,df_this_team$player_9,df_this_team$player_10,df_this_team$player_11))-1
    
    team_id_data_frame[team_id_data_frame$team_api_id %in% team_id_data_frame[i,1],"players_fielded"] <- count
   
    i = i+1
}

Merge Data

In [259]:
team_data_frame <- merge(team_data_frame,team_id_data_frame,by="team_api_id",all.x=TRUE)

Write the csv

In [260]:
write.table(player_data_frame,file="/home/opensam/Programs/jupyter/lab2/player.csv",sep=",")
write.table(team_data_frame,file="/home/opensam/Programs/jupyter/lab2/team.csv",sep=",")